In [13]:
%matplotlib inline
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, LSTM, Input

import tensorflow as tf

import collections
import numpy as np
import sys

print("Tensorflow version: {}".format(tf.__version__))
print("Keras version: {}".format(keras.__version__))

Tensorflow version: 1.3.0
Keras version: 2.0.8


In [14]:
# load ascii text and covert to lowercase
raw_text = open("./data/001ssb.txt").read()
raw_text = raw_text.lower()

In [15]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)

print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  1607894
Total Vocab:  53


In [16]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 30
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)

print("Total Patterns: ", n_patterns)

Total Patterns:  1607864


In [17]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(n_vocab)

# one hot encode the output variable
y = keras.utils.to_categorical(dataY)

In [18]:
# define the LSTM model
inp = Input(shape=(X.shape[1], X.shape[2]))
x = LSTM(256, return_sequences=True)(inp)
x = Dropout(0.2)(x)
x = LSTM(256)(x)
x = Dropout(0.2)(x)
output = Dense(y.shape[1], activation ='softmax')(x)

generative_model = Model(inputs=inp, outputs=output)

optimizer = keras.optimizers.RMSprop(lr=0.01)
generative_model.compile(loss='categorical_crossentropy', optimizer='adam')

generative_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 30, 1)             0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 30, 256)           264192    
_________________________________________________________________
dropout_5 (Dropout)          (None, 30, 256)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 53)                13621     
Total params: 803,125
Trainable params: 803,125
Non-trainable params: 0
_________________________________________________________________


In [19]:
# fit the model
generative_model.fit(X, y, epochs=20, batch_size=64)

Epoch 1/20
1607864/1607864 [==============================] - 14003s - loss: 2.2311 
Epoch 2/20
 166272/1607864 [==>...........................] - ETA: 13326s - loss: 1.9575

KeyboardInterrupt: 

In [26]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = generative_model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" rmed his belly. he held out th "
e sooth of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of the soote of